In [263]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import openpyxl
import datetime as dt
import pandas as pd
from sklearn import preprocessing
from sklearn import preprocessing

In [264]:
file = r'dealcloselistdata.csv'
file2 = r'team_detail.csv'
file3 = r'mandate_rejection_percentage.xlsx'
file4 = r'emplevel_data_from2019.xlsx'
data = pd.read_csv(file)
data2 = pd.read_csv(file2)
data3 = pd.read_excel(file3)
data4 = pd.read_excel(file4)

In [268]:
ddd = data4[['years', 'months', 'empcode', 'collection', 'emptype', 'empname',
       'empcity', 'ecs_dealclosed', 'ecs_hits','ecs_collection', 'active_employee',
       'presentdays', 'absentese_percentage', 'appintment_given',
       'appointment_attended', 'added_on', 'realisable_value',
       'realisable_value_1_yr', 'ccsi_amt', 'total_dealclose', 'ecs_dealclose',
       'bounce_dealclose', 'enach_dealclose', 'digital_dealclose',
       'ecs_deal_percent', 'enach_deal_percent', 'digital_deal_percent',
       'budget', 'date_of_leaving', 'active_flag', 
       'mandate_rejection_perc', 'total_mandate', 'rejected_mandate']]

In [269]:
df2=data.join(data3, lsuffix="_left", rsuffix="_right")
df2 = df2.drop(columns=['years_right','months_right','new_empcode'])
df2.rename(columns= {'years_left':'years','months_left':'months','empcode_left':'empcode'},inplace=True)

label_encoder = preprocessing.LabelEncoder()
ddd['emptype'] = label_encoder.fit_transform(ddd['emptype'])

ddd['present_percent'] = ddd['presentdays'] / ddd['active_employee'] * 100

In [274]:
ddd['years'].value_counts()

2019    73168
2020    24344
Name: years, dtype: int64

In [280]:
DD = ddd.groupby(['empcode','emptype']).agg(month_count = ('months','count'),total_working_days = ('active_employee','sum'),total_present_days = ('presentdays','sum'),present_percent = ('present_percent','mean'),collection_sum = ('collection','sum'),ecs_sum = ('ecs_hits','sum'),ecs_down_payment = ('ecs_collection','sum'), total_dc_sum =('total_dealclose','sum'), ecs_dc_sum = ('ecs_dealclose','sum'), total_mandate = ('total_mandate','sum'), total_mandate_rejected = ('rejected_mandate','sum')) 
DD.reset_index(inplace=True)


DD['mandate_clear_percent'] = (DD['total_mandate']-DD['total_mandate_rejected'])/DD['total_mandate']*100
DD['ecs_dc_percentage'] = DD['ecs_dc_sum']/DD['total_dc_sum']*100
DD['Total_revenue'] = DD['collection_sum'] + DD['ecs_sum']
DD['Total_revenue_per_month'] = DD['Total_revenue']/DD['month_count']

data = pd.merge(DD, ddd[['empcode','empname','date_of_leaving','added_on']], on ='empcode')
data.drop_duplicates(inplace=True)

In [281]:
DD.loc[(DD['empcode']==19)]
# "{:.2f}".format(DD['total_present_days'])

,empcode,emptype,month_count,total_working_days,total_present_days,present_percent,collection_sum,ecs_sum,ecs_down_payment,total_dc_sum,ecs_dc_sum,total_mandate,total_mandate_rejected,mandate_clear_percent,ecs_dc_percentage,Total_revenue,Total_revenue_per_month
4,19,1,12,305,300,98.333333,8.267123e+06,1.457965e+06,493139.830508,128.0,35.0,46,11,76.086957,27.34375,9.725088e+06,810424.013418


In [282]:
data.loc[(data['empcode']==19)]

,empcode,emptype,month_count,total_working_days,total_present_days,present_percent,collection_sum,ecs_sum,ecs_down_payment,total_dc_sum,ecs_dc_sum,total_mandate,total_mandate_rejected,mandate_clear_percent,ecs_dc_percentage,Total_revenue,Total_revenue_per_month,empname,date_of_leaving,added_on
13,19,1,12,305,300,98.333333,8.267123e+06,1.457965e+06,493139.830508,128.0,35.0,46,11,76.086957,27.34375,9.725088e+06,810424.013418,Anjana Parmar,2021-02-06 00:00:00,1997-04-17 00:00:00


In [289]:
DD.loc[(DD['empcode']==17702)]

,empcode,emptype,month_count,total_working_days,total_present_days,present_percent,collection_sum,ecs_sum,ecs_down_payment,total_dc_sum,ecs_dc_sum,total_mandate,total_mandate_rejected,mandate_clear_percent,ecs_dc_percentage,Total_revenue,Total_revenue_per_month
465,17702,2,12,306,277,90.519448,1.731184e+07,8.266133e+06,765690.254237,446.0,96.5,115,12,89.565217,21.636771,2.557798e+07,2.131498e+06


#### TME

In [334]:
TME = data.loc[(data['emptype']==2)]
TME_old = TME[['empcode','emptype','month_count','total_working_days','total_present_days','total_mandate', 'total_mandate_rejected', 'collection_sum','ecs_sum','total_dc_sum','ecs_dc_sum','ecs_dc_percentage','present_percent','mandate_clear_percent','Total_revenue','Total_revenue_per_month']]

TME_old['per_Day_Revenue_3y'] = TME_old['Total_revenue']/TME_old['total_working_days']
TME_old.fillna(0,inplace=True)

x = TME_old.iloc[0:,7:].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled, index=TME_old.empcode[0:])

TME_old.reset_index(inplace=True)
df.reset_index(inplace=True)

bbb = pd.concat([df.iloc[:,1:],TME_old], axis=1)

bbb.rename(columns= {0:'Norm_collection_sum',1:'Norm_ecs_sum',2:'Norm_total_dc_sum',3:'Norm_ecs_dc_sum',4:'Norm_ecs_dc_percentage',5:'Norm_present_percent',6:'Norm_mandate_clear_percent',7:'Norm_Total_revenue',8:'Norm_Total_revenue_per_month', 9:'Norm_per_Day_Revenue_3y'}, inplace=True)
bbb.fillna(0,inplace=True)


a = bbb['Norm_present_percent']*20
b = bbb['Norm_mandate_clear_percent']*20
c = bbb['Norm_ecs_dc_percentage']*20
d = bbb['Norm_per_Day_Revenue_3y']*20
e = bbb['Norm_mandate_clear_percent']*20

bbb['Performance']= a+b+c+d+e
bbb['grade'] = 0

a=bbb
for index in a.index:
    if a.loc[index,'Performance']>=70.00:
        a.loc[index,'grade'] = 1
    elif (a.loc[index,'Performance']>=60.00) & (a.loc[index,'Performance']<70.00):
        a.loc[index,'grade'] = 2
    elif (a.loc[index,'Performance']>=40.00) & (a.loc[index,'Performance']<60.00):
        a.loc[index,'grade'] = 3
    elif (a.loc[index,'Performance']>=10.00) & (a.loc[index,'Performance']<40.00):
        a.loc[index,'grade'] = 4
    elif (a.loc[index,'Performance']>=0.00) & (a.loc[index,'Performance']<10.00):
        a.loc[index,'grade'] = 5

TME_new2 = a
TME_new2.columns
TME_new2.set_index('empcode',inplace=True)

TME_new2.drop(columns=['index'],inplace=True)
TME_new2.reset_index(inplace=True)

TME_2 = TME_new2[['empcode', 'emptype','month_count','collection_sum', 
'ecs_sum', 'total_dc_sum','ecs_dc_sum','ecs_dc_percentage','total_working_days',
'total_present_days', 'present_percent','Total_revenue','Total_revenue_per_month', 'per_Day_Revenue_3y',  'total_mandate', 
'total_mandate_rejected','mandate_clear_percent','Performance', 'grade']]


TME = pd.merge(df2[['empcode','empname','empcity','date_of_leaving','added_on']],TME_2, on='empcode')
TME.drop_duplicates(inplace=True)


TME.fillna(0,inplace=True)
TME['emptype'] =label_encoder.inverse_transform(TME['emptype'])

TME_1_active = TME.loc[(TME['grade']==1) & (TME['month_count']>=1) & (TME['date_of_leaving']==0) | (TME['grade']==1) & (TME['month_count']>=1) & (TME['date_of_leaving']=='0000-00-00 00:00:00')] 
TME_2_active = TME.loc[(TME['grade']==2) & (TME['month_count']>=1) & (TME['date_of_leaving']==0) | (TME['grade']==2) & (TME['month_count']>=1) & (TME['date_of_leaving']=='0000-00-00 00:00:00')] 
TME_3_active = TME.loc[(TME['grade']==3) & (TME['month_count']>=1) & (TME['date_of_leaving']==0) | (TME['grade']==3) & (TME['month_count']>=1) & (TME['date_of_leaving']=='0000-00-00 00:00:00')] 
TME_4_active = TME.loc[(TME['grade']==4) & (TME['month_count']>=1) & (TME['date_of_leaving']==0) | (TME['grade']==4) & (TME['month_count']>=1) & (TME['date_of_leaving']=='0000-00-00 00:00:00')] 
TME_5_active = TME.loc[(TME['grade']==5) & (TME['month_count']>=1) & (TME['date_of_leaving']==0) | (TME['grade']==5) & (TME['month_count']>=1) & (TME['date_of_leaving']=='0000-00-00 00:00:00')] 

TME_active = pd.concat([TME_1_active, TME_2_active, TME_3_active,TME_4_active, TME_5_active], axis=0)
TME_active.reset_index(inplace=True)

TME_active.fillna(0,inplace=True)
TME_active.drop(columns='index',inplace=True)


TME_active = TME_active[['empcode', 'empname', 'empcity', 'date_of_leaving', 'added_on',
       'emptype', 'month_count', 'collection_sum', 'ecs_sum', 'total_dc_sum',
       'ecs_dc_sum', 'ecs_dc_percentage','total_working_days',
       'total_present_days', 'present_percent',
       'Total_revenue', 'Total_revenue_per_month', 'per_Day_Revenue_3y', 'total_mandate', 'total_mandate_rejected',
       'mandate_clear_percent' ,'Performance','grade']]

TME = TME_active[['collection_sum', 'ecs_sum', 'total_dc_sum',
       'ecs_dc_sum', 'ecs_dc_percentage', 'total_working_days',
       'total_present_days', 'present_percent', 'Total_revenue',
       'Total_revenue_per_month', 'per_Day_Revenue_3y','total_mandate', 'total_mandate_rejected',
       'mandate_clear_percent', 'Performance','grade']].round(decimals = 2)

TME1 = TME_active[['empcode', 'empname', 'empcity', 'date_of_leaving', 'added_on',
       'emptype', 'month_count']]


TME_final = pd.concat([TME1,TME], axis=1)

TME_super30 = TME_final.loc[(TME_final['per_Day_Revenue_3y']>500) & (TME_final['month_count']>=12)]
TME_rising_superstars = TME_final.loc[(TME_final['month_count']<12) & (TME_final['per_Day_Revenue_3y']>500)]
TME_super30 = TME_super30.sort_values(by='Performance',ascending=False).head(30).reset_index()
TME_rising_superstars = TME_rising_superstars.sort_values(by=('Performance'),ascending=False).head(30).reset_index()
TME_super30.drop(columns='index',inplace=True)
TME_rising_superstars.drop(columns='index',inplace=True)

/tmp/ipykernel_7232/1653166767.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TME_old['per_Day_Revenue_3y'] = TME_old['Total_revenue']/TME_old['total_working_days']
/tmp/ipykernel_7232/1653166767.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TME_old.fillna(0,inplace=True)


In [335]:
TME_final.loc[(TME_final['empcode']==17702)]
# data.loc[(data['empcode']==19)]

,empcode,empname,empcity,date_of_leaving,added_on,emptype,month_count,collection_sum,ecs_sum,total_dc_sum,...,total_present_days,present_percent,Total_revenue,Total_revenue_per_month,per_Day_Revenue_3y,total_mandate,total_mandate_rejected,mandate_clear_percent,Performance,grade
61,17702,Krishna Choudhari,Noida,0000-00-00 00:00:00,2011-04-04 00:00:00,TME,12,17311843.3,8266132.63,446.0,...,277,90.52,25577975.92,2131497.99,83588.16,115,12,89.57,64.01,2


In [343]:
TME_super30.to_csv('TME_super30.csv')
TME_rising_superstars.to_csv('TME_rising_superstars.csv')

In [336]:
TME_super30

,empcode,empname,empcity,date_of_leaving,added_on,emptype,month_count,collection_sum,ecs_sum,total_dc_sum,...,total_present_days,present_percent,Total_revenue,Total_revenue_per_month,per_Day_Revenue_3y,total_mandate,total_mandate_rejected,mandate_clear_percent,Performance,grade
0,9082,Abhishek Singh,Mumbai,0000-00-00 00:00:00,2009-04-24 00:00:00,TME,12,6771355.08,53680560.17,203.5,...,263,86.11,60451915.25,5037659.60,198203.00,137,26,81.02,77.76,1
1,10000597,BHARATHI D,Bangalore,0000-00-00 00:00:00,2012-03-19 00:00:00,TME,12,11267045.44,49670850.00,218.0,...,277,91.10,60937895.44,5078157.95,200453.60,45,7,84.44,72.61,1
2,10040018,Suchitra Swapnil Kudalkar,Pune,0000-00-00 00:00:00,2016-05-02 00:00:00,TME,12,933755.08,10926041.10,97.5,...,274,89.97,11859796.19,988316.35,39012.49,105,10,90.48,70.07,1
3,10002738,Sayali Phate,Pune,0000-00-00 00:00:00,2012-06-15 00:00:00,TME,12,884709.75,4452938.98,76.5,...,282,92.77,5337648.73,444804.06,17558.06,93,9,90.32,69.08,2
4,10619,Lajwanti Saraf,Pune,0000-00-00 00:00:00,2009-09-08 00:00:00,TME,12,646123.31,4325518.22,55.5,...,282,92.88,4971641.53,414303.46,16354.08,47,2,95.74,67.92,2
5,9222,Seema Gaddam,Pune,0000-00-00 00:00:00,2009-05-05 00:00:00,TME,12,1231204.40,5708295.76,53.5,...,283,93.16,6939500.16,578291.68,22827.30,54,6,88.89,67.82,2
6,23056,Deepali Dhimdhime,Pune,0000-00-00 00:00:00,2012-01-17 00:00:00,TME,12,636897.03,4417521.19,44.0,...,277,91.17,5054418.22,421201.52,16626.38,36,2,94.44,67.57,2
7,10021129,Kiran Parekh,Ahmedabad,0000-00-00 00:00:00,2014-05-12 00:00:00,TME,12,906302.67,4166855.51,87.5,...,290,95.10,5073158.18,422763.18,16633.31,101,12,88.12,67.38,2
8,10003305,GAYATHRI PAUL,Bangalore,0000-00-00 00:00:00,2012-07-10 00:00:00,TME,12,1717649.58,14422470.34,108.0,...,243,79.91,16140119.92,1345009.99,53092.50,88,12,86.36,67.06,2
9,10032764,Jyoti Patil,Pune,0000-00-00 00:00:00,2015-07-13 00:00:00,TME,12,1624125.42,2476202.54,177.5,...,271,89.28,4100327.97,341694.00,13487.92,156,13,91.67,66.67,2


In [260]:
TME_rising_superstars

,empcode,empname,empcity,date_of_leaving,added_on,emptype,month_count,collection_sum,ecs_sum,total_dc_sum,...,total_present_days,present_percent,Total_revenue,Total_revenue_per_month,per_Day_Revenue_3y,total_mandate,total_mandate_rejected,mandate_clear_percent,Performance,grade
0,8389,DEBALINA DUTTA GUPTA,Kolkata,0000-00-00 00:00:00,2009-02-16 00:00:00,TME,1,50000.00,0.00,1.0,...,24,96.00,50000.00,50000.00,2000.00,1,0,100.00,70.82,1
1,10085409,kanak,Noida,0000-00-00 00:00:00,2019-12-02 00:00:00,TME,2,26999.15,0.00,7.0,...,44,88.00,26999.15,13499.58,539.98,4,0,100.00,68.92,2
2,10038942,Soumyadip Sarkar,Kolkata,0000-00-00 00:00:00,2016-03-28 00:00:00,TME,4,35715.25,117036.44,3.0,...,84,81.52,152751.69,38187.92,1483.03,3,0,100.00,67.66,2
3,10083817,Keerthana venkatesh,Coimbatore,0000-00-00 00:00:00,2019-10-09 00:00:00,TME,4,48081.36,77788.98,10.0,...,102,99.00,125870.34,31467.58,1222.04,11,1,90.91,66.60,2
4,10084283,K KIRAN PRASATH,Coimbatore,0000-00-00 00:00:00,2019-11-05 00:00:00,TME,4,110245.76,998521.19,16.0,...,98,97.00,1108766.95,277191.74,10977.89,15,1,93.33,66.58,2
5,10088387,Kirti Maheshwari,Noida,0,2020-01-29 00:00:00,TME,2,44786.44,0.00,13.5,...,42,84.00,44786.44,22393.22,895.73,12,0,100.00,66.26,2
6,10084030,Rupali Rajendra Chavan,Mumbai,0000-00-00 00:00:00,2019-10-15 00:00:00,TME,6,87117.80,6000.00,16.5,...,134,87.74,93117.80,15519.63,612.62,17,0,100.00,65.85,2
7,10088153,stephygraph pandi,Coimbatore,0000-00-00 00:00:00,2020-01-27 00:00:00,TME,1,32400.00,0.00,6.0,...,23,92.00,32400.00,32400.00,1296.00,5,0,100.00,65.71,2
8,10086425,Neeta govindrao khandare,Pune,0000-00-00 00:00:00,2019-12-17 00:00:00,TME,3,49366.10,6093.22,19.5,...,73,96.10,55459.32,18486.44,729.73,24,3,87.50,65.64,2
9,10088253,Arpitha M P,Bangalore,0,2020-01-27 00:00:00,TME,2,29669.49,4513.56,8.0,...,48,94.08,34183.05,17091.53,670.26,8,1,87.50,63.99,2


#### JDA

In [337]:
JDA = data.loc[(data['emptype']==0)]
JDA_old = JDA[['empcode','emptype','month_count','total_working_days','total_present_days','total_mandate', 'total_mandate_rejected', 'collection_sum','ecs_sum','total_dc_sum','ecs_dc_sum','ecs_dc_percentage','present_percent','mandate_clear_percent','Total_revenue','Total_revenue_per_month']]

JDA_old['per_Day_Revenue_3y'] = JDA_old['Total_revenue']/JDA_old['total_working_days']
JDA_old.fillna(0,inplace=True)

x = JDA_old.iloc[0:,7:].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled, index=JDA_old.empcode[0:])

JDA_old.reset_index(inplace=True)
df.reset_index(inplace=True)

bbb = pd.concat([df.iloc[:,1:],JDA_old], axis=1)

bbb.rename(columns= {0:'Norm_collection_sum',1:'Norm_ecs_sum',2:'Norm_total_dc_sum',3:'Norm_ecs_dc_sum',4:'Norm_ecs_dc_percentage',5:'Norm_present_percent',6:'Norm_mandate_clear_percent',7:'Norm_Total_revenue',8:'Norm_Total_revenue_per_month', 9:'Norm_per_Day_Revenue_3y'}, inplace=True)
bbb.fillna(0,inplace=True)


a = bbb['Norm_present_percent']*20
b = bbb['Norm_mandate_clear_percent']*20
c = bbb['Norm_ecs_dc_percentage']*20
d = bbb['Norm_per_Day_Revenue_3y']*20
e = bbb['Norm_mandate_clear_percent']*20

bbb['Performance']= a+b+c+d+e
bbb['grade'] = 0

a=bbb
for index in a.index:
    if a.loc[index,'Performance']>=70.00:
        a.loc[index,'grade'] = 1
    elif (a.loc[index,'Performance']>=60.00) & (a.loc[index,'Performance']<70.00):
        a.loc[index,'grade'] = 2
    elif (a.loc[index,'Performance']>=40.00) & (a.loc[index,'Performance']<60.00):
        a.loc[index,'grade'] = 3
    elif (a.loc[index,'Performance']>=10.00) & (a.loc[index,'Performance']<40.00):
        a.loc[index,'grade'] = 4
    elif (a.loc[index,'Performance']>=0.00) & (a.loc[index,'Performance']<10.00):
        a.loc[index,'grade'] = 5

JDA_new2 = a
JDA_new2.columns
JDA_new2.set_index('empcode',inplace=True)

JDA_new2.drop(columns=['index'],inplace=True)
JDA_new2.reset_index(inplace=True)

JDA_2 = JDA_new2[['empcode', 'emptype','month_count','collection_sum', 
'ecs_sum', 'total_dc_sum','ecs_dc_sum','ecs_dc_percentage','total_working_days',
'total_present_days', 'present_percent','Total_revenue','Total_revenue_per_month', 'per_Day_Revenue_3y',  'total_mandate', 
'total_mandate_rejected','mandate_clear_percent','Performance', 'grade']]


JDA = pd.merge(df2[['empcode','empname','empcity','date_of_leaving','added_on']],JDA_2, on='empcode')
JDA.drop_duplicates(inplace=True)


JDA.fillna(0,inplace=True)
JDA['emptype'] =label_encoder.inverse_transform(JDA['emptype'])

JDA_1_active = JDA.loc[(JDA['grade']==1) & (JDA['month_count']>=1) & (JDA['date_of_leaving']==0) | (JDA['grade']==1) & (JDA['month_count']>=1) & (JDA['date_of_leaving']=='0000-00-00 00:00:00')] 
JDA_2_active = JDA.loc[(JDA['grade']==2) & (JDA['month_count']>=1) & (JDA['date_of_leaving']==0) | (JDA['grade']==2) & (JDA['month_count']>=1) & (JDA['date_of_leaving']=='0000-00-00 00:00:00')] 
JDA_3_active = JDA.loc[(JDA['grade']==3) & (JDA['month_count']>=1) & (JDA['date_of_leaving']==0) | (JDA['grade']==3) & (JDA['month_count']>=1) & (JDA['date_of_leaving']=='0000-00-00 00:00:00')] 
JDA_4_active = JDA.loc[(JDA['grade']==4) & (JDA['month_count']>=1) & (JDA['date_of_leaving']==0) | (JDA['grade']==4) & (JDA['month_count']>=1) & (JDA['date_of_leaving']=='0000-00-00 00:00:00')] 
JDA_5_active = JDA.loc[(JDA['grade']==5) & (JDA['month_count']>=1) & (JDA['date_of_leaving']==0) | (JDA['grade']==5) & (JDA['month_count']>=1) & (JDA['date_of_leaving']=='0000-00-00 00:00:00')] 

JDA_active = pd.concat([JDA_1_active, JDA_2_active, JDA_3_active,JDA_4_active, JDA_5_active], axis=0)
JDA_active.reset_index(inplace=True)

JDA_active.fillna(0,inplace=True)
JDA_active.drop(columns='index',inplace=True)


JDA_active = JDA_active[['empcode', 'empname', 'empcity', 'date_of_leaving', 'added_on',
       'emptype', 'month_count', 'collection_sum', 'ecs_sum', 'total_dc_sum',
       'ecs_dc_sum', 'ecs_dc_percentage','total_working_days',
       'total_present_days', 'present_percent',
       'Total_revenue', 'Total_revenue_per_month', 'per_Day_Revenue_3y', 'total_mandate', 'total_mandate_rejected',
       'mandate_clear_percent' ,'Performance','grade']]

JDA = JDA_active[['collection_sum', 'ecs_sum', 'total_dc_sum',
       'ecs_dc_sum', 'ecs_dc_percentage', 'total_working_days',
       'total_present_days', 'present_percent', 'Total_revenue',
       'Total_revenue_per_month', 'per_Day_Revenue_3y','total_mandate', 'total_mandate_rejected',
       'mandate_clear_percent', 'Performance','grade']].round(decimals = 2)

JDA1 = JDA_active[['empcode', 'empname', 'empcity', 'date_of_leaving', 'added_on',
       'emptype', 'month_count']]


JDA_final = pd.concat([JDA1,JDA], axis=1)

JDA_super30 = JDA_final.loc[(JDA_final['per_Day_Revenue_3y']>500) & (JDA_final['month_count']>=12)]
JDA_rising_superstars = JDA_final.loc[(JDA_final['month_count']<12) & (JDA_final['per_Day_Revenue_3y']>500)]
JDA_super30 = JDA_super30.sort_values(by='Performance',ascending=False).head(30).reset_index()
JDA_rising_superstars = JDA_rising_superstars.sort_values(by=('Performance'),ascending=False).head(30).reset_index()
JDA_super30.drop(columns='index',inplace=True)
JDA_rising_superstars.drop(columns='index',inplace=True)

/tmp/ipykernel_7232/3198013358.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  JDA_old['per_Day_Revenue_3y'] = JDA_old['Total_revenue']/JDA_old['total_working_days']
/tmp/ipykernel_7232/3198013358.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  JDA_old.fillna(0,inplace=True)


In [338]:
JDA_super30

,empcode,empname,empcity,date_of_leaving,added_on,emptype,month_count,collection_sum,ecs_sum,total_dc_sum,...,total_present_days,present_percent,Total_revenue,Total_revenue_per_month,per_Day_Revenue_3y,total_mandate,total_mandate_rejected,mandate_clear_percent,Performance,grade
0,10045115,Daram Mahendra Reddy,Hyderabad,0000-00-00 00:00:00,2016-10-10 00:00:00,JDA,12,2352248.31,4365700.42,182.0,...,294,96.71,6717948.73,559829.06,22098.52,129,8,93.80,75.06,1
1,10024462,Sundar Srivastav,Noida,0000-00-00 00:00:00,2014-09-29 00:00:00,JDA,12,5707846.61,1588024.58,184.5,...,302,98.57,7295871.19,607989.27,23842.72,90,5,94.44,73.91,1
2,10023068,Bhushan Bhure,Pune,0000-00-00 00:00:00,2014-06-24 00:00:00,JDA,12,823047.46,2889208.05,128.5,...,290,95.40,3712255.51,309354.63,12211.37,149,28,81.21,70.08,1
3,10022806,Souvik Ghosh,Kolkata,0,2014-06-24 00:00:00,JDA,12,975120.34,790080.08,121.0,...,296,96.78,1765200.42,147100.04,5768.63,95,8,91.58,69.88,2
4,10043178,Sukhdew Mandal,Pune,0,2016-07-25 00:00:00,JDA,12,1037868.22,2317731.36,165.5,...,288,94.70,3355599.58,279633.30,11038.16,120,9,92.50,69.78,2
5,10047289,Pankaj Sonke,Pune,0,2017-01-09 00:00:00,JDA,12,377250.85,293234.75,97.0,...,267,88.19,670485.59,55873.80,2205.54,80,5,93.75,69.22,2
6,10065822,Somashekhar M N,Bangalore,0,2018-06-25 00:00:00,JDA,12,979933.90,377705.93,143.0,...,289,95.15,1357639.83,113136.65,4465.92,106,7,93.40,68.85,2
7,10051786,Dharavath Venkanna,Hyderabad,0,2017-06-05 00:00:00,JDA,12,2104332.63,2420494.07,152.5,...,291,95.79,4524826.69,377068.89,14884.30,121,19,84.30,68.71,2
8,10062371,Boora Ravi Kumar,Hyderabad,0000-00-00 00:00:00,2018-04-18 00:00:00,JDA,12,1542705.08,1139587.29,93.5,...,285,93.88,2682292.37,223524.36,8823.33,71,7,90.14,68.32,2
9,10071961,Nikhil Sharma,Noida,0000-00-00 00:00:00,2019-01-07 00:00:00,JDA,12,637953.39,63356.78,80.0,...,288,94.10,701310.17,58442.51,2291.86,66,7,89.39,68.29,2


In [339]:
JDA_rising_superstars

,empcode,empname,empcity,date_of_leaving,added_on,emptype,month_count,collection_sum,ecs_sum,total_dc_sum,...,total_present_days,present_percent,Total_revenue,Total_revenue_per_month,per_Day_Revenue_3y,total_mandate,total_mandate_rejected,mandate_clear_percent,Performance,grade
0,10015114,Abhee Omar,Pune,0000-00-00 00:00:00,2013-10-07 00:00:00,JDA,1,127118.64,0.00,1.0,...,22,88.00,127118.64,127118.64,5084.75,5,0,100.00,77.76,1
1,10006694,Sachin Arane,Mumbai,0000-00-00 00:00:00,2012-10-20 00:00:00,JDA,5,2742349.58,3319133.05,90.0,...,122,94.40,6061482.64,1212296.53,46988.24,24,2,91.67,75.63,1
2,10023076,Sahil Khan,Pune,0000-00-00 00:00:00,2014-06-24 00:00:00,JDA,11,535660.17,526541.53,53.5,...,275,98.65,1062201.69,96563.79,3807.17,68,4,94.12,73.39,1
3,10088223,Bodhisattwa Purkait,Kolkata,0,2020-01-27 00:00:00,JDA,2,42777.12,1198.31,15.0,...,42,82.62,43975.42,21987.71,862.26,15,0,100.00,72.34,1
4,10085189,Ravi Gupta,Noida,0,2019-11-26 00:00:00,JDA,4,110800.00,4500.00,15.0,...,99,97.08,115300.00,28825.00,1130.39,13,1,92.31,71.94,1
5,10084725,venkatachalapathy,Coimbatore,0,2019-11-15 00:00:00,JDA,4,118114.41,20775.42,34.0,...,99,98.04,138889.83,34722.46,1375.15,30,2,93.33,71.77,1
6,10085803,Ravi Kumar,Noida,0,2019-12-09 00:00:00,JDA,4,141600.00,0.00,15.0,...,87,85.54,141600.00,35400.00,1388.24,13,0,100.00,71.75,1
7,10084735,Riyad N,Coimbatore,0,2019-11-18 00:00:00,JDA,4,91567.80,0.00,19.5,...,92,91.39,91567.80,22891.95,906.61,22,0,100.00,71.46,1
8,10087204,Gaurav Sharma,Noida,0000-00-00 00:00:00,2020-01-13 00:00:00,JDA,2,42491.53,1247.46,10.0,...,42,84.00,43738.98,21869.49,874.78,8,0,100.00,71.26,1
9,10041265,Akhil T Ramanan,Coimbatore,0000-00-00 00:00:00,2016-06-09 00:00:00,JDA,3,78701.69,0.00,6.5,...,77,101.23,78701.69,26233.90,1035.55,4,0,100.00,70.69,1


In [344]:
JDA_super30.to_csv('JDA_super30.csv')
JDA_rising_superstars.to_csv('JDA_rising_superstars.csv')

#### ME

In [340]:
ME = data.loc[(data['emptype']==1)]
ME_old = ME[['empcode','emptype','month_count','total_working_days','total_present_days','total_mandate', 'total_mandate_rejected', 'collection_sum','ecs_sum','total_dc_sum','ecs_dc_sum','ecs_dc_percentage','present_percent','mandate_clear_percent','Total_revenue','Total_revenue_per_month']]

ME_old['per_Day_Revenue_3y'] = ME_old['Total_revenue']/ME_old['total_working_days']
ME_old.fillna(0,inplace=True)

x = ME_old.iloc[0:,7:].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled, index=ME_old.empcode[0:])

ME_old.reset_index(inplace=True)
df.reset_index(inplace=True)

bbb = pd.concat([df.iloc[:,1:],ME_old], axis=1)

bbb.rename(columns= {0:'Norm_collection_sum',1:'Norm_ecs_sum',2:'Norm_total_dc_sum',3:'Norm_ecs_dc_sum',4:'Norm_ecs_dc_percentage',5:'Norm_present_percent',6:'Norm_mandate_clear_percent',7:'Norm_Total_revenue',8:'Norm_Total_revenue_per_month', 9:'Norm_per_Day_Revenue_3y'}, inplace=True)
bbb.fillna(0,inplace=True)


a = bbb['Norm_present_percent']*20
b = bbb['Norm_mandate_clear_percent']*20
c = bbb['Norm_ecs_dc_percentage']*20
d = bbb['Norm_per_Day_Revenue_3y']*20
e = bbb['Norm_mandate_clear_percent']*20

bbb['Performance']= a+b+c+d+e
bbb['grade'] = 0

a=bbb
for index in a.index:
    if a.loc[index,'Performance']>=70.00:
        a.loc[index,'grade'] = 1
    elif (a.loc[index,'Performance']>=60.00) & (a.loc[index,'Performance']<70.00):
        a.loc[index,'grade'] = 2
    elif (a.loc[index,'Performance']>=40.00) & (a.loc[index,'Performance']<60.00):
        a.loc[index,'grade'] = 3
    elif (a.loc[index,'Performance']>=10.00) & (a.loc[index,'Performance']<40.00):
        a.loc[index,'grade'] = 4
    elif (a.loc[index,'Performance']>=0.00) & (a.loc[index,'Performance']<10.00):
        a.loc[index,'grade'] = 5

ME_new2 = a
ME_new2.columns
ME_new2.set_index('empcode',inplace=True)

ME_new2.drop(columns=['index'],inplace=True)
ME_new2.reset_index(inplace=True)

ME_2 = ME_new2[['empcode', 'emptype','month_count','collection_sum', 
'ecs_sum', 'total_dc_sum','ecs_dc_sum','ecs_dc_percentage','total_working_days',
'total_present_days', 'present_percent','Total_revenue','Total_revenue_per_month', 'per_Day_Revenue_3y',  'total_mandate', 
'total_mandate_rejected','mandate_clear_percent','Performance', 'grade']]


ME = pd.merge(df2[['empcode','empname','empcity','date_of_leaving','added_on']],ME_2, on='empcode')
ME.drop_duplicates(inplace=True)


ME.fillna(0,inplace=True)
ME['emptype'] =label_encoder.inverse_transform(ME['emptype'])

ME_1_active = ME.loc[(ME['grade']==1) & (ME['month_count']>=1) & (ME['date_of_leaving']==0) | (ME['grade']==1) & (ME['month_count']>=1) & (ME['date_of_leaving']=='0000-00-00 00:00:00')] 
ME_2_active = ME.loc[(ME['grade']==2) & (ME['month_count']>=1) & (ME['date_of_leaving']==0) | (ME['grade']==2) & (ME['month_count']>=1) & (ME['date_of_leaving']=='0000-00-00 00:00:00')] 
ME_3_active = ME.loc[(ME['grade']==3) & (ME['month_count']>=1) & (ME['date_of_leaving']==0) | (ME['grade']==3) & (ME['month_count']>=1) & (ME['date_of_leaving']=='0000-00-00 00:00:00')] 
ME_4_active = ME.loc[(ME['grade']==4) & (ME['month_count']>=1) & (ME['date_of_leaving']==0) | (ME['grade']==4) & (ME['month_count']>=1) & (ME['date_of_leaving']=='0000-00-00 00:00:00')] 
ME_5_active = ME.loc[(ME['grade']==5) & (ME['month_count']>=1) & (ME['date_of_leaving']==0) | (ME['grade']==5) & (ME['month_count']>=1) & (ME['date_of_leaving']=='0000-00-00 00:00:00')] 

ME_active = pd.concat([ME_1_active, ME_2_active, ME_3_active,ME_4_active, ME_5_active], axis=0)
ME_active.reset_index(inplace=True)

ME_active.fillna(0,inplace=True)
ME_active.drop(columns='index',inplace=True)


ME_active = ME_active[['empcode', 'empname', 'empcity', 'date_of_leaving', 'added_on',
       'emptype', 'month_count', 'collection_sum', 'ecs_sum', 'total_dc_sum',
       'ecs_dc_sum', 'ecs_dc_percentage','total_working_days',
       'total_present_days', 'present_percent',
       'Total_revenue', 'Total_revenue_per_month', 'per_Day_Revenue_3y', 'total_mandate', 'total_mandate_rejected',
       'mandate_clear_percent' ,'Performance','grade']]

ME = ME_active[['collection_sum', 'ecs_sum', 'total_dc_sum',
       'ecs_dc_sum', 'ecs_dc_percentage', 'total_working_days',
       'total_present_days', 'present_percent', 'Total_revenue',
       'Total_revenue_per_month', 'per_Day_Revenue_3y','total_mandate', 'total_mandate_rejected',
       'mandate_clear_percent', 'Performance','grade']].round(decimals = 2)

ME1 = ME_active[['empcode', 'empname', 'empcity', 'date_of_leaving', 'added_on',
       'emptype', 'month_count']]


ME_final = pd.concat([ME1,ME], axis=1)

ME_super30 = ME_final.loc[(ME_final['per_Day_Revenue_3y']>500) & (ME_final['month_count']>=12)]
ME_rising_superstars = ME_final.loc[(ME_final['month_count']<12) & (ME_final['per_Day_Revenue_3y']>500)]
ME_super30 = ME_super30.sort_values(by='Performance',ascending=False).head(30).reset_index()
ME_rising_superstars = ME_rising_superstars.sort_values(by=('Performance'),ascending=False).head(30).reset_index()
ME_super30.drop(columns='index',inplace=True)
ME_rising_superstars.drop(columns='index',inplace=True)

/tmp/ipykernel_7232/871416539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ME_old['per_Day_Revenue_3y'] = ME_old['Total_revenue']/ME_old['total_working_days']
/tmp/ipykernel_7232/871416539.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ME_old.fillna(0,inplace=True)


In [341]:
ME_super30

,empcode,empname,empcity,date_of_leaving,added_on,emptype,month_count,collection_sum,ecs_sum,total_dc_sum,...,total_present_days,present_percent,Total_revenue,Total_revenue_per_month,per_Day_Revenue_3y,total_mandate,total_mandate_rejected,mandate_clear_percent,Performance,grade
0,2129,Jayantrao Gavhane,Pune,0000-00-00 00:00:00,2007-03-10 00:00:00,ME,12,5902390.31,21716181.36,269.0,...,317,104.27,27618571.67,2301547.64,90850.56,157,15,90.45,69.88,2
1,10014610,Navin Kumar,Pune,0000-00-00 00:00:00,2013-09-16 00:00:00,ME,12,3545234.75,30457874.58,246.0,...,314,103.27,34003109.32,2833592.44,111852.33,164,27,83.54,68.46,2
2,650,Ganesh Mulik,Pune,0000-00-00 00:00:00,2005-11-08 00:00:00,ME,12,2426563.23,2472716.10,38.0,...,294,96.66,4899279.33,408273.28,16116.05,33,1,96.97,67.74,2
3,10001648,Navin Bedadur,Pune,0000-00-00 00:00:00,2012-05-04 00:00:00,ME,12,2210333.05,10025564.41,189.0,...,306,100.64,12235897.46,1019658.12,40249.66,148,19,87.16,67.57,2
4,10008670,Dheepakkumar A,Chennai,0000-00-00 00:00:00,2013-01-07 00:00:00,ME,12,12569702.47,24261906.78,481.0,...,306,100.44,36831609.25,3069300.77,120759.37,119,6,94.96,67.02,2
5,10064187,Pratik Mandlik,Pune,0000-00-00 00:00:00,2018-05-28 00:00:00,ME,12,1720729.66,729775.42,185.0,...,312,102.64,2450505.08,204208.76,8060.87,123,10,91.87,66.47,2
6,10004118,Dhananjay Bhanage,Pune,0000-00-00 00:00:00,2012-08-04 00:00:00,ME,12,3156149.30,12320329.66,186.0,...,307,100.95,15476478.96,1289706.58,50909.47,113,13,88.50,66.40,2
7,10018205,Swapnil Misar,Pune,0,2014-01-27 00:00:00,ME,12,2133626.27,2595992.37,186.0,...,304,99.97,4729618.64,394134.89,15557.96,89,2,97.75,66.36,2
8,10029281,Pradeep Patil M,Bangalore,0000-00-00 00:00:00,2015-04-13 00:00:00,ME,12,3423108.47,6586354.24,298.0,...,300,98.74,10009462.71,834121.89,32925.86,215,26,87.91,66.35,2
9,20433,Naveen Kumar,Bangalore,0000-00-00 00:00:00,2011-08-26 00:00:00,ME,12,4002839.83,5615197.46,255.5,...,304,100.06,9618037.29,801503.11,31638.28,121,6,95.04,66.07,2


In [342]:
ME_rising_superstars

,empcode,empname,empcity,date_of_leaving,added_on,emptype,month_count,collection_sum,ecs_sum,total_dc_sum,...,total_present_days,present_percent,Total_revenue,Total_revenue_per_month,per_Day_Revenue_3y,total_mandate,total_mandate_rejected,mandate_clear_percent,Performance,grade
0,10015427,Nishant Gupta,Bangalore,0000-00-00 00:00:00,2013-10-01 00:00:00,ME,1,24314.41,0.00,7.0,...,25,100.00,24314.41,24314.41,972.58,16,0,100.00,77.60,1
1,6665656565665,sdcsdcsd,Mumbai,0000-00-00 00:00:00,2013-05-29 00:00:00,ME,5,568910.17,0.00,105.0,...,129,100.00,568910.17,113782.03,4410.16,37,0,100.00,76.47,1
2,10071668,Jeetoo Tiwari,Pune,0,2018-12-26 00:00:00,ME,1,6433.05,12580.51,4.0,...,19,76.00,19013.56,19013.56,760.54,7,0,100.00,73.38,1
3,23267,Amit Kumar,Pune,0000-00-00 00:00:00,2012-02-01 00:00:00,ME,7,449307.86,1326559.32,19.0,...,167,95.07,1775867.19,253695.31,10090.15,22,2,90.91,71.52,1
4,10077963,Sachin Ganesh Lande,Pune,0000-00-00 00:00:00,2019-05-28 00:00:00,ME,3,165183.05,69814.41,35.0,...,80,105.23,234997.46,78332.49,3092.07,23,0,100.00,71.21,1
5,10089074,Riyaj Salim Maner,Pune,0,2020-02-17 00:00:00,ME,1,17300.00,0.00,3.0,...,25,100.00,17300.00,17300.00,692.00,2,0,100.00,70.92,1
6,10015044,Moseen Syed,Pune,0000-00-00 00:00:00,2013-09-30 00:00:00,ME,2,512937.29,1586804.24,48.0,...,57,114.00,2099741.53,1049870.76,41994.83,29,1,96.55,70.65,1
7,10086251,Ravi kumar,Noida,0,2019-12-16 00:00:00,ME,3,193369.49,72650.85,49.0,...,80,105.13,266020.34,88673.45,3500.27,55,9,83.64,70.47,1
8,10083643,Sarath Kumar,Coimbatore,0000-00-00 00:00:00,2019-09-24 00:00:00,ME,5,507803.39,63423.73,67.0,...,119,93.85,571227.12,114245.42,4497.85,51,1,98.04,70.27,1
9,10062996,Ajinkya Shrirao,Pune,0,2018-04-30 00:00:00,ME,1,27005.93,88496.61,10.0,...,25,100.00,115502.54,115502.54,4620.10,13,1,92.31,69.74,2


In [345]:
ME_super30.to_csv('ME_super30.csv')
ME_rising_superstars.to_csv('ME_rising_superstars.csv')